# Iteración 2 - CNN Básica con Data Augmentation

Mismo modelo que la iteración 1 pero ahora añadimos data augmentation para intentar mejorar la generalización.

**Arquitectura**: 3 capas Conv (32→64→128) + Dense(512) + Dense(2)

**Cambios**: Añadido RandomFlip, RandomRotation, RandomZoom y aumento de épocas a 20

In [ ]:
# Imports básicos
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras

seed = 42
keras.utils.set_random_seed(seed)

DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Versión de Keras:", keras.__version__)

In [ ]:
# Cargo datos con split 80/20 y modo categorical (todavía no arreglado el bug)
image_size = (256, 256)
batch_size = 32

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
)

print(f"Training batches: {len(train_ds)}")
print(f"Validation batches: {len(val_ds)}")

In [ ]:
# Añado data augmentation para ver si mejoro
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
], name="data_augmentation")

print("Data Augmentation configurado")

In [ ]:
# Construyo el modelo igual que antes pero con data augmentation
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = image_size + (3,)

model = Sequential([
    keras.Input(shape=input_shape),
    data_augmentation,
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
], name='CNN_con_DataAugmentation')

model.summary()

In [ ]:
# Compilo y entreno con más épocas
%%time

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

epochs = 20  # Más épocas porque data augmentation necesita más tiempo

print(f"Épocas: {epochs}")
print(f"Optimizer: RMSprop")
print("-" * 60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1
)

print("-" * 60)
print(f"Val Accuracy final: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
# Pinto las curvas de entrenamiento
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))

plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:, "loss"], lw=2, label='Pérdida en entrenamiento')
plt.plot(logs.loc[1:, "val_loss"], lw=2, label='Pérdida en validación')
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:, "accuracy"], lw=2, label='Precisión en entrenamiento')
plt.plot(logs.loc[1:, "val_accuracy"], lw=2, label='Precisión en validación')
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPrecisión final en entrenamiento: {logs['accuracy'].iloc[-1]:.4f}")
print(f"Precisión final en validación: {logs['val_accuracy'].iloc[-1]:.4f}")

In [ ]:
# Guardo el modelo
model.save("model.keras")
print("Modelo guardado como 'model.keras'")

In [ ]:
# Evalúo con supplementary para ver si mejora
supplementary_ds = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
)

print("Evaluando con datos suplementarios...")
results = model.evaluate(supplementary_ds, return_dict=True, verbose=1)

print(f"\nSupplementary Accuracy: {results['accuracy']:.4f}")

In [ ]:
# Genero predicciones para test
%%time

predictions_dict = {}

print(f"Generando predicciones para {len(os.listdir(TEST_PATH))} imágenes...")

for img in os.listdir(TEST_PATH):
    img_path = os.path.join(TEST_PATH, img)
    file_name = img_path.split('/')[-1]
    file_no_extension = file_name.split('.')[0]
    
    img_loaded = keras.utils.load_img(img_path, target_size=image_size)
    img_array = keras.utils.img_to_array(img_loaded)
    img_array = keras.ops.expand_dims(img_array, 0)
    
    prediction = model.predict(img_array, verbose=0)
    label = int(keras.ops.argmax(prediction[0]).numpy())
    
    predictions_dict[int(file_no_extension)] = label

print(f"Predicciones completadas: {len(predictions_dict)}")

In [ ]:
# Creo el submission
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"])
submission = submission.sort_values(by='id', ascending=True)
submission.to_csv('submission.csv', index=False)

print("Archivo de submission creado")
print("\nDistribución de predicciones:")
print(submission["label"].value_counts())
print(f"\nTotal: {len(submission)} imágenes")